In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, MaxPooling2D, Flatten, Dropout, BatchNormalization, Reshape, Conv2D,ReLU, Bidirectional
import pandas as pd
import numpy as np
import gc
import random

In [ ]:
def clear_variable(var_list):
    for var in var_list:
        globals().pop(var, None)
    gc.collect()

In [ ]:
subset_list = []

for i in range(20):
  subset_list.append(f'Shuffled_Subset{i+1}.h5')

In [ ]:
X_train_list, X_test_list, X_val_list = [], [], []
y_train_list, y_test_list, y_val_list = [], [], []

for i in range(len(subset_list)):
  file_path = f'Shuffled_Subset{i+1}.h5'
  read_data = pd.read_hdf(file_path)

  data_reshaped = np.array(read_data.drop('Class', axis=1)).reshape(read_data.shape[0],30900,4)
  data_labels = read_data['Class']
  clear_variable('read_data')

  X_train_list.append(data_reshaped[:700])
  X_val_list.append(data_reshaped[700:800])
  X_test_list.append(data_reshaped[800:])
  clear_variable('data_reshaped')

  y_train_list.append(data_labels[:700])
  y_val_list.append(data_labels[700:800])
  y_test_list.append(data_labels[800:])
  clear_variable('data_labels')

X_train = np.concatenate(X_train_list, axis=0)
clear_variable('X_train_list')

X_test = np.concatenate(X_test_list, axis=0)
clear_variable('X_test_list')

X_val = np.concatenate(X_val_list, axis=0)
clear_variable('X_val_list')

y_train = np.concatenate(y_train_list, axis=0)
clear_variable('y_train_list')

y_test = np.concatenate(y_test_list, axis=0)
clear_variable('y_test_list')

y_val = np.concatenate(y_val_list, axis=0)
clear_variable('y_val_list')

## **Deep Learning Models**

In [ ]:
num_classes = 8
input_shape = (30900, 4)

In [ ]:
import numpy as np
from tensorflow.keras.utils import Sequence

class DataGenerator(Sequence):
    def __init__(self, X, y, batch_size):
        self.X = X
        self.y = y
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.X) / self.batch_size))

    def __getitem__(self, idx):
        batch_x = self.X[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        return np.array(batch_x), np.array(batch_y)


In [ ]:
from tensorflow.keras.utils import Sequence
import numpy as np

class CustomDataGenerator(Sequence):
    def __init__(self, data, labels, batch_size, shuffle=True):
        self.data = data
        self.labels = labels
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indexes = np.arange(len(self.data))

    def __len__(self):
        return int(np.floor(len(self.data) / self.batch_size))

    def __getitem__(self, index):
        # Get the batch indexes
        batch_indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        batch_data = self.data[batch_indexes]
        batch_labels = self.labels[batch_indexes]

        return batch_data, batch_labels

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indexes)


In [ ]:
from tensorflow.keras.callbacks import LearningRateScheduler

def scheduler(epoch, lr):
    if epoch < 5:
        return lr
    else:
        return float(lr * tf.math.exp(-0.1).numpy())

lr_scheduler = LearningRateScheduler(scheduler)


### **CNN Model**

In [ ]:
# Build the model
model = Sequential([
    Reshape((input_shape[0], input_shape[1], 1), input_shape=input_shape),

    Conv2D(filters=32, kernel_size=(5, 5), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(4, 4)),

    Conv2D(filters=64, kernel_size=(5, 1), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(4, 1)),

    Conv2D(filters=128, kernel_size=(5, 1), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(4, 1)),

    Flatten(),

    Dense(units=256, activation='relu'),
    Dropout(0.5),

    Dense(units=num_classes, activation='softmax')
])


# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Model summary
model.summary()


### **CNN-LSTM Model**

In [ ]:
# Build the model
model = Sequential([
    Reshape((input_shape[0], input_shape[1], 1), input_shape=input_shape),

    Conv2D(filters=32, kernel_size=(5, 5), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(4, 4)),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(5, 1), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(4, 1)),
    Dropout(0.3),

    Conv2D(filters=128, kernel_size=(5, 1), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(4, 1)),
    Dropout(0.3),

    Reshape((-1, 128)),

    Bidirectional(LSTM(units=64, activation='tanh')),
    BatchNormalization(),
    Dropout(0.3),

    Dense(units=256, activation='relu'),
    Dropout(0.5),

    Dense(units=num_classes, activation='softmax')
])



# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Model summary
model.summary()


In [ ]:
# Assuming data is split into train, val, test sets
train_gen = DataGenerator(X_train, y_train, batch_size=32)
val_gen = DataGenerator(X_val, y_val, batch_size=32)

# Now train the model
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=25,
    callbacks=[lr_scheduler],
    verbose=1
)

In [ ]:
test_gen = DataGenerator(X_test, y_test, batch_size=64)
test_loss, test_acc = model.evaluate(test_gen)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_acc}")

In [ ]:
# Save the related model
model.save('models/one-hot_cnn_lstm.keras')
model.save('models/one-hot_cnn.keras')